In [754]:
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta
from pandas import IndexSlice as idx

In [755]:
techlog_raw = pd.read_csv("csv_data_files\TECHLOG.csv")
techlog_raw['DATE'] = pd.to_datetime(techlog_raw['DATE'], format="%Y-%m-%d")
techlog_raw['ATA'] = techlog_raw['ATA'].astype("category")

util_raw = pd.read_csv("./csv_data_files/UTIL.csv")
util_raw['TO_DATETIME'] = pd.to_datetime(util_raw['TO_DATE'] + ' ' + util_raw['TO_TIME (UTC)'])
util_raw['LAND_DATETIME'] = pd.to_datetime(util_raw['LAND_DATE'] + ' ' + util_raw['LAND_TIME (UTC)'])
util_raw['FL_DURR'] = util_raw['LAND_DATETIME'] - util_raw['TO_DATETIME']
util_raw.drop(columns=['TO_DATE', 'TO_TIME (UTC)', 'LAND_DATE', 'LAND_TIME (UTC)'], inplace=True)

In [756]:
pirep_tbl = techlog_raw[techlog_raw['PM'] == "PIREP"].groupby(['ATA',pd.Grouper(key='DATE', freq='M')])[['PM']].count()
pirep_tbl

PM
ATA DATE          
    2022-04-30   0
    2022-05-31   0
    2022-06-30   0
0   2022-04-30  12
    2022-05-31   9
    2022-06-30  41
21  2022-04-30   0
    2022-05-31   0
    2022-06-30   3
25  2022-04-30   0
    2022-05-31   0
    2022-06-30   0
27  2022-04-30   2
    2022-05-31   0
    2022-06-30   0
33  2022-04-30   0
    2022-05-31   0
    2022-06-30   0
34  2022-04-30   0
    2022-05-31   0
    2022-06-30   0
36  2022-04-30   2
    2022-05-31   0
    2022-06-30   0
52  2022-04-30   0
    2022-05-31   0
    2022-06-30   0

In [757]:
mon_hrs = util_raw.groupby(pd.Grouper(key='TO_DATETIME', freq='M'))[['FL_DURR']].sum()
mon_hrs['FL_DURR'] = mon_hrs['FL_DURR']/np.timedelta64(1,'s')/3600
mon_hrs

,FL_DURR
TO_DATETIME,
2022-04-30,64.783333
2022-05-31,50.166667
2022-06-30,115.283333


In [758]:
last12M = (datetime.now() - relativedelta(months=+12)).strftime('%Y-%m-%d')
thisM = datetime.now().strftime('%Y-%m-%d')
last12M_filter = idx[:,last12M:thisM]
ata_index = pirep_tbl.index.get_level_values(0).unique()
date_index = pirep_tbl.index.get_level_values(1).unique()
period = len(pirep_tbl.loc[last12M_filter].index.get_level_values(1).unique())

In [759]:
pirrate_list = []
for ind in ata_index:
    for d_ind in date_index:
        pirrate_list.extend(pirep_tbl.loc[idx[ind,d_ind]]['PM'] / (mon_hrs.loc[d_ind]/100))

pirep_tbl['PIR_RATE'] = pirrate_list

In [760]:
pirep_tbl

PM   PIR_RATE
ATA DATE                     
    2022-04-30   0   0.000000
    2022-05-31   0   0.000000
    2022-06-30   0   0.000000
0   2022-04-30  12  18.523283
    2022-05-31   9  17.940199
    2022-06-30  41  35.564551
21  2022-04-30   0   0.000000
    2022-05-31   0   0.000000
    2022-06-30   3   2.602284
25  2022-04-30   0   0.000000
    2022-05-31   0   0.000000
    2022-06-30   0   0.000000
27  2022-04-30   2   3.087214
    2022-05-31   0   0.000000
    2022-06-30   0   0.000000
33  2022-04-30   0   0.000000
    2022-05-31   0   0.000000
    2022-06-30   0   0.000000
34  2022-04-30   0   0.000000
    2022-05-31   0   0.000000
    2022-06-30   0   0.000000
36  2022-04-30   2   3.087214
    2022-05-31   0   0.000000
    2022-06-30   0   0.000000
52  2022-04-30   0   0.000000
    2022-05-31   0   0.000000
    2022-06-30   0   0.000000

In [761]:
# Sum of PIREP per ATA for 12-month period
sum_pirep = [pirep_tbl.loc[idx[ind,last12M:thisM],'PM'].sum() for ind in ata_index]
sum_pirep = pd.Series(sum_pirep, index=ata_index)

# Sum of PIREP rate per ATA for the last 12 months
sum_pirrate = [pirep_tbl.loc[idx[ind,last12M:thisM],'PIR_RATE'].sum() for ind in ata_index]
sum_pirrate = pd.Series(sum_pirrate, index=ata_index)

In [762]:
# Mean PIREPS of each ATA for 12-month period
mean = sum_pirrate / period
mean 

ATA
       0.000000
0     24.009344
21     0.867428
25     0.000000
27     1.029071
33     0.000000
34     0.000000
36     1.029071
52     0.000000
dtype: float64

In [763]:
sq_diff = []

for ind in ata_index:
    sq_diff.extend((pirep_tbl.loc[ind,'PIR_RATE'].values - mean.loc[ind])**2)

pirep_tbl['sq_diff'] = sq_diff
pirep_tbl


PM   PIR_RATE     sq_diff
ATA DATE                                 
    2022-04-30   0   0.000000    0.000000
    2022-05-31   0   0.000000    0.000000
    2022-06-30   0   0.000000    0.000000
0   2022-04-30  12  18.523283   30.096872
    2022-05-31   9  17.940199   36.834522
    2022-06-30  41  35.564551  133.522802
21  2022-04-30   0   0.000000    0.752431
    2022-05-31   0   0.000000    0.752431
    2022-06-30   3   2.602284    3.009726
25  2022-04-30   0   0.000000    0.000000
    2022-05-31   0   0.000000    0.000000
    2022-06-30   0   0.000000    0.000000
27  2022-04-30   2   3.087214    4.235951
    2022-05-31   0   0.000000    1.058988
    2022-06-30   0   0.000000    1.058988
33  2022-04-30   0   0.000000    0.000000
    2022-05-31   0   0.000000    0.000000
    2022-06-30   0   0.000000    0.000000
34  2022-04-30   0   0.000000    0.000000
    2022-05-31   0   0.000000    0.000000
    2022-06-30   0   0.000000    0.000000
36  2022-04-30   2   3.087214    4.235951
    2022-05-31   0   0.000000    1.058988
    2022-06-30   0   0.000000    1.058988
52  2022-04-30   0   0.000000    0.000000
    2022-05-31   0   0.000000    0.000000
    2022-06-30   0   0.000000    0.000000

In [764]:
ucl_tbl = pd.DataFrame(dict(
    ATA=['21','22','23','24','25','26','27','28','29','30','31','32','33','34','35','36','38','46','49','51','52','54','56','70','71','73','74','75','77'], 
    Description=['AIR CONDITIONING', 'AUTO FLIGHT', 'COMMUNICATIONS', 'ELECTRICAL POWER', 'EQUIPMENT/FURNISHING', 'FIRE PROTECTION', 'FLIGHT CONTROLS', 'FUEL', 'HYDRAULIC POWER', 'ICE & RAIN PROTECTION', 'INDICATING / RECORDING SYSTEMS', 'LANDING GEAR', 'LIGHTS', 'NAVIGATION', 'OXYGEN', 'PNEUMATIC', 'WATER / WASTE', 'INFORMATION SYSTEMS', 'AIRBORNE AUXILIARY POWER', 'STANDARDS PRACTICES & STRUCTURE', 'DOORS', 'NACELLES / PYLONS', 'WINDOWS', ' STANDARD PRACTICES & ENGINE', 'POWER PLANT', 'ENGINE FUEL & CONTROL', 'IGNITION', 'ENGINE AIR', 'ENGINE INDICATING'], 
    PIREPs=[None]*29,
    PIRRATE_12mth=[None]*29,
    UCL=['NaN']*29)).set_index('ATA')


In [765]:
for ind in ata_index: 
    ucl = np.sqrt(pirep_tbl.loc[ind,'sq_diff'].sum() / (period-1))
    ucl_tbl['PIREPs'].loc[ind] = sum_pirep.loc[ind]
    ucl_tbl['PIRRATE_12mth'] = ucl_tbl['PIREPs'] / mon_hrs[last12M:thisM]['FL_DURR'].sum()
    ucl_tbl['UCL'].loc[ind] = ucl

In [766]:
# The UCL values per ATA chapter, taking into account last 12-month PIREP rate
ucl_tbl

,Description,PIREPs,PIRRATE_12mth,UCL
ATA,,,,
21,AIR CONDITIONING,3,0.01303,1.502429
22,AUTO FLIGHT,None,NaN,NaN
23,COMMUNICATIONS,None,NaN,NaN
24,ELECTRICAL POWER,None,NaN,NaN
25,EQUIPMENT/FURNISHING,0,0.0,0.0
26,FIRE PROTECTION,None,NaN,NaN
27,FLIGHT CONTROLS,2,0.008687,1.782404
28,FUEL,None,NaN,NaN
29,HYDRAULIC POWER,None,NaN,NaN
